# Split apply and combine

In [1]:
import addutils.toc ; addutils.toc.js(ipy_notebook=True)

In [2]:
from utilities.generators import generate_all
generate_all()

Generating p06_d2.txt...
Skipped p03_AAPL.csv (already existing)
Skipped p03_DAX.csv (already existing)
Skipped p07_d1.txt (already existing)
Skipped p01_d4.csv (already existing)
Skipped p01_volumes.txt (already existing)
Skipped p07_portfolio.h5 (already existing)
Skipped p06_d3.txt (already existing)
Skipped p01_d2.csv (already existing)
Skipped p01_prices.txt (already existing)
Skipped p07_d2.csv (already existing)
Skipped baby_names/ (already existing)
Skipped p01_d3.csv (already existing)


In this tutorial we are going to see advanced data management with `pandas` data structures.

In [3]:
import numpy as np
import pandas as pd
from IPython.display import (display, HTML)
from addutils import side_by_side2
from addutils import css_notebook
css_notebook()

Categorizing a data set and applying a function to each group, is often a critical component of a data analysis workflow. After
loading, merging, and preparing a data set, a familiar task is to compute group statistics or possibly pivot tables for reporting or visualization purposes. *pandas* provides a flexible and high-performance groupby facility.

By *'group by'* we refer to a process involving one or more of the following steps:

* **Splitting** the data into groups based on some criteria
* **Applying** a function to each group independently
* **Combining** the results into a data structure

Suppose we are managing a website and we have a log-file with number of *wiews* and *likes* coming from different cities:

In [4]:
d1 = pd.read_csv('temp/p07_d1.txt', index_col=0)
d1 = d1.reindex(columns=['State','City','Views','Likes'])
display(d1)

,State,City,Views,Likes
0,NE,Page,10,4
1,KY,Stone,9,3
2,CO,Rye,3,0
3,CO,Rye,7,2
4,KY,Dema,4,1
5,KY,Keavy,2,1
6,CO,Rye,1,0
7,NE,Cairo,8,3
8,CO,Dumont,12,7


## 1 Groupby

`groupby` groups DataFrame or Series by a parameter on a given axis:

In [5]:
g1 = d1.groupby('State')
print g1.groups

{'KY': [1, 4, 5], 'CO': [2, 3, 6, 8], 'NE': [0, 7]}


The variable `groups` of a `GroupBy` object is a dictionary containing indexes of each group member.

In [6]:
for name,group in g1:
    print name
    print group
    print 'Total Views: %d - Total Likes: %d\n\n' %(group['Views'].sum(),
                                                    group['Likes'].sum())

CO
  State    City  Views  Likes
2    CO     Rye      3      0
3    CO     Rye      7      2
6    CO     Rye      1      0
8    CO  Dumont     12      7
Total Views: 23 - Total Likes: 9


KY
  State   City  Views  Likes
1    KY  Stone      9      3
4    KY   Dema      4      1
5    KY  Keavy      2      1
Total Views: 15 - Total Likes: 5


NE
  State   City  Views  Likes
0    NE   Page     10      4
7    NE  Cairo      8      3
Total Views: 18 - Total Likes: 7




It is also possibile to apply a `groupby` over a hierarchical index `DataFrame`

In [7]:
d2 = d1.set_index(['State','City'])
display(d2)

Views  Likes
State City                
NE    Page       10      4
KY    Stone       9      3
CO    Rye         3      0
      Rye         7      2
KY    Dema        4      1
      Keavy       2      1
CO    Rye         1      0
NE    Cairo       8      3
CO    Dumont     12      7

## 2 Aggregate

Once the GroupBy object has been created, several methods are available to perform a computation on the grouped data. Here we use `aggregate`. The result of the aggregation will have the group names as the new index along the grouped axis. In the case of multiple keys, the result is a MultiIndex by default, though this can be changed by using the `as_index option`:

In [8]:
g2 = d2.groupby(level=[0])
print g2.groups
g2.aggregate(np.sum)

{'KY': [('KY', 'Stone'), ('KY', 'Dema'), ('KY', 'Keavy')], 'CO': [('CO', 'Rye'), ('CO', 'Rye'), ('CO', 'Rye'), ('CO', 'Dumont')], 'NE': [('NE', 'Page'), ('NE', 'Cairo')]}


,Views,Likes
State,,
CO,23,9
KY,15,5
NE,18,7


In [9]:
g3 = d2.groupby(level=[0,1])
g4 = d2.groupby(level=[0,1], as_index=False)
HTML(side_by_side2(g3.aggregate(np.sum), g4.aggregate(np.sum)))

`aggregate` allows to pass any function that returns a scalar value from a vector and can handle list of functions:

In [10]:
d1[['State', 'Views']].groupby('State').aggregate([np.sum, np.mean, np.std])

Views                
        sum  mean       std
State                      
CO       23  5.75  4.856267
KY       15  5.00  3.605551
NE       18  9.00  1.414214

## 3 Apply

`apply` will extend the previous concepts to any Python function:

In [11]:
pd.set_option('display.float_format', lambda x: '{:.1f}'.format(x))

def add_field(group):
    group['Tot.Views'] = group['Views'].sum()
    group['Likes[%]'] = 100.0*group['Likes']/group['Likes'].sum()
    return group

HTML(side_by_side2(d1, d1.groupby('State').apply(add_field)))

## 4 A pratical example: Normalize by year

In [12]:
idx = pd.date_range('1999/5/28', periods=1500, freq='1B')
s1 = pd.Series(np.random.normal(5.5, 2, 1500), idx)
s1 = pd.rolling_mean(s1, 10, 10).dropna()

Here we define a grouping key for months and one for years:

In [13]:
def my_groupby_key_year(timestamp):
    return timestamp.year

def my_groupby_key_month(timestamp):
    return timestamp.month

def my_normalization(group):
    return (group-group.mean())/group.std()

Here we normalize the data on a monthly base and check mean and std on an yearly base:

In [14]:
t1 = s1.groupby(my_groupby_key_month).apply(my_normalization)

HTML(side_by_side2(s1.head(8),
                   t1.head(8),
                   t1.groupby(my_groupby_key_year).aggregate([np.mean, np.std])))

,0
1999-06-10,5.1
1999-06-11,5.0
1999-06-14,5.2
1999-06-15,5.2
1999-06-16,5.5
1999-06-17,5.4
1999-06-18,5.6
1999-06-21,5.5
,0
1999-06-10,-0.6


## 5 A practical example: Group and standardize by dimension

In [15]:
d3 = pd.read_csv('example_data/company.csv', index_col=0)
display(d3.head())

,Value,expenses,employees,Dimension
Company,,,,
Dapibus Company,96 008,7124,78,Big
Pede Blandit Congue Company,61 562,8454,60,Big
Pede Suspendisse Associates,54 728,6641,4,Small
Dictum Associates,16 802,6498,89,Big
Dui Cras Pellentesque Ltd,93 954,5040,97,Big


Since the column "Value" is made by strings with a space separator we need a simpel intermediate step to convert values from string to floats:

In [16]:
d3['Value'] = d3['Value'].apply(lambda x: float(x.replace(' ', '')))
d3.head()

,Value,expenses,employees,Dimension
Company,,,,
Dapibus Company,96008.0,7124,78,Big
Pede Blandit Congue Company,61562.0,8454,60,Big
Pede Suspendisse Associates,54728.0,6641,4,Small
Dictum Associates,16802.0,6498,89,Big
Dui Cras Pellentesque Ltd,93954.0,5040,97,Big


In [17]:
d3.groupby('Dimension').mean()

,Value,expenses,employees
Dimension,,,
Big,49445.4,5474.9,61.1
Small,60947.8,4521.4,31.8


---

Visit [www.add-for.com](<http://www.add-for.com/IT>) for more tutorials and updates.

This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.